<a href="https://colab.research.google.com/github/P4rad0x47/-1/blob/main/%D0%9B%D0%90%D0%91%D0%90_%D0%98%D0%93%D0%A0%D0%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from abc import ABC, abstractmethod

class Ability:
    """Класс для способностей/умений"""
    def __init__(self, name: str, damage: int, mana_cost: int, cooldown: int = 0):
        self.name = name
        self.damage = damage
        self.mana_cost = mana_cost
        self.cooldown = cooldown
        self.current_cooldown = 0

    def is_ready(self) -> bool:
        """Проверяет, готово ли умение к использованию"""
        return self.current_cooldown == 0

    def use(self):
        """Использование умения (устанавливает кулдаун)"""
        if self.cooldown > 0:
            self.current_cooldown = self.cooldown

    def reduce_cooldown(self):
        """Уменьшает кулдаун на 1 ход"""
        if self.current_cooldown > 0:
            self.current_cooldown -= 1

    def __str__(self):
        return f"{self.name} (урон: {self.damage}, мана: {self.mana_cost}, кулдаун: {self.cooldown})"

class Item:
    """Класс для предметов"""
    def __init__(self, name: str, item_type: str, value: int, durability: int = 100):
        self.name = name
        self.type = item_type  # weapon, armor, potion, etc.
        self.value = value  # damage for weapons, armor for armor, heal for potions
        self.durability = durability

    def use(self):
        """Использование предмета (уменьшает прочность)"""
        if self.durability > 0:
            self.durability -= 10
        return self.value if self.durability > 0 else 0

    def __str__(self):
        return f"{self.name} ({self.type}, значение: {self.value}, прочность: {self.durability}%)"

class Character(ABC):
    """Базовый класс для всех персонажей"""
    def __init__(self, name: str, health: int, mana: int, attack: int, defense: int):
        self.name = name
        self.max_health = health
        self.health = health
        self.max_mana = mana
        self.mana = mana
        self.attack = attack
        self.defense = defense
        self.abilities = []
        self.inventory = []
        self.level = 1
        self.experience = 0

    def add_ability(self, ability: Ability):
        """Добавляет способность персонажу"""
        self.abilities.append(ability)

    def add_item(self, item: Item):
        """Добавляет предмет в инвентарь"""
        self.inventory.append(item)

    def use_item(self, item_index: int):
        """Использует предмет из инвентаря"""
        if 0 <= item_index < len(self.inventory):
            item = self.inventory[item_index]
            effect = item.use()

            if item.type == "potion":
                self.health = min(self.max_health, self.health + effect)
                print(f"{self.name} использовал {item.name} и восстановил {effect} здоровья!")

            if item.durability <= 0:
                self.inventory.pop(item_index)

    def basic_attack(self) -> int:
        """Базовая атака (возвращает урон)"""
        return max(1, self.attack - random.randint(0, 3))

    def take_damage(self, damage: int):
        """Получение урона с учетом защиты"""
        actual_damage = max(1, damage - self.defense // 2)
        self.health -= actual_damage
        return actual_damage

    def is_alive(self) -> bool:
        """Проверяет, жив ли персонаж"""
        return self.health > 0

    def restore_stats(self):
        """Восстанавливает здоровье и ману (после боя)"""
        self.health = self.max_health
        self.mana = self.max_mana
        for ability in self.abilities:
            ability.current_cooldown = 0

    def reduce_cooldowns(self):
        """Уменьшает кулдауны всех способностей"""
        for ability in self.abilities:
            ability.reduce_cooldown()

    def check_level_up(self):
        """Проверяет, нужно ли повысить уровень"""
        if self.experience >= self.level * 100:
            self.level += 1
            self.experience = 0
            self.max_health += 10
            self.max_mana += 5
            self.attack += 2
            self.defense += 1
            self.health = self.max_health
            self.mana = self.max_mana
            print(f"\n{self.name} повысил уровень до {self.level}!")
            print(f"Характеристики улучшены: Здоровье +10, Мана +5, Атака +2, Защита +1")

    @abstractmethod
    def special_ability(self):
        """Специальная способность класса"""
        pass

    def __str__(self):
        return f"{self.name} (Ур. {self.level}) - Здоровье: {self.health}/{self.max_health}, Мана: {self.mana}/{self.max_mana}"

class Warrior(Character):
    """Класс воина"""
    def __init__(self, name: str):
        super().__init__(name, health=120, mana=30, attack=15, defense=10)
        self.add_ability(Ability("Мощный удар", 25, 15, 2))

    def special_ability(self):
        """Специальная способность воина - мощный удар"""
        if self.mana >= 15:
            ability = self.abilities[0]
            if ability.is_ready():
                self.mana -= 15
                damage = ability.damage + random.randint(0, 5)
                ability.use()
                return damage
        return 0

class Mage(Character):
    """Класс мага"""
    def __init__(self, name: str):
        super().__init__(name, health=70, mana=100, attack=8, defense=5)
        self.add_ability(Ability("Огненный шар", 35, 25, 3))
        self.add_ability(Ability("Ледяная стрела", 20, 15, 1))

    def special_ability(self, ability_index: int = 0):
        """Специальная способность мага - заклинания"""
        if 0 <= ability_index < len(self.abilities):
            ability = self.abilities[ability_index]
            if self.mana >= ability.mana_cost and ability.is_ready():
                self.mana -= ability.mana_cost
                ability.use()
                return ability.damage + random.randint(0, 3)
        return 0

class Rogue(Character):
    """Класс разбойника"""
    def __init__(self, name: str):
        super().__init__(name, health=90, mana=50, attack=12, defense=7)
        self.add_ability(Ability("Критический удар", 40, 20, 4))
        self.crit_chance = 0.3

    def special_ability(self):
        """Специальная способность разбойника - критический удар"""
        if self.mana >= 20:
            ability = self.abilities[0]
            if ability.is_ready() and random.random() < self.crit_chance:
                self.mana -= 20
                ability.use()
                return ability.damage
        return 0

    def basic_attack(self) -> int:
        """Базовая атака с шансом на крит"""
        base_damage = super().basic_attack()
        if random.random() < self.crit_chance:
            base_damage *= 2
            print(f"{self.name} наносит критический удар!")
        return base_damage

class Paladin(Character):
    """Класс паладина"""
    def __init__(self, name: str):
        super().__init__(name, health=110, mana=60, attack=10, defense=12)
        self.add_ability(Ability("Божественное исцеление", -30, 25, 3))
        self.add_ability(Ability("Кара небес", 25, 20, 2))

    def special_ability(self, ability_index: int = 0):
        """Специальная способность паладина - исцеление или атака"""
        if 0 <= ability_index < len(self.abilities):
            ability = self.abilities[ability_index]
            if self.mana >= ability.mana_cost and ability.is_ready():
                self.mana -= ability.mana_cost
                ability.use()

                if ability.damage < 0:  # исцеление
                    heal_amount = -ability.damage
                    self.health = min(self.max_health, self.health + heal_amount)
                    return -heal_amount  # отрицательное значение для исцеления
                else:  # атака
                    return ability.damage + random.randint(0, 2)
        return 0

class Enemy(Character):
    """Базовый класс врагов"""
    def __init__(self, name: str, health: int, attack: int, defense: int, exp_reward: int):
        super().__init__(name, health, 0, attack, defense)
        self.exp_reward = exp_reward

    def special_ability(self):
        """Враги используют способности случайным образом"""
        if self.abilities and random.random() < 0.3:
            ability = random.choice(self.abilities)
            if ability.is_ready():
                ability.use()
                return ability.damage
        return 0

    def take_damage(self, damage: int):
        """Получение урона с учетом защиты"""
        actual_damage = max(1, damage - self.defense // 3)
        self.health -= actual_damage
        return actual_damage

class Goblin(Enemy):
    """Класс гоблина"""
    def __init__(self):
        super().__init__("Гоблин", health=40, attack=8, defense=3, exp_reward=50)
        self.add_ability(Ability("Грязный удар", 12, 0, 2))

class Orc(Enemy):
    """Класс орка"""
    def __init__(self):
        super().__init__("Орк", health=80, attack=12, defense=6, exp_reward=100)
        self.add_ability(Ability("Размашистый удар", 20, 0, 3))

class Dragon(Enemy):
    """Класс дракона"""
    def __init__(self):
        super().__init__("Дракон", health=150, attack=18, defense=12, exp_reward=300)
        self.add_ability(Ability("Огненное дыхание", 35, 0, 4))

class Battle:
    """Класс для управления боем"""
    def __init__(self, player: Character, enemy: Enemy):
        self.player = player
        self.enemy = enemy
        self.turn = 0

    def start(self):
        """Начинает бой"""
        print(f"\n=== Бой начинается: {self.player.name} vs {self.enemy.name} ===")
        print(f"{self.player}\n{self.enemy}\n")

        while self.player.is_alive() and self.enemy.is_alive():
            self.turn += 1
            print(f"\n--- Ход {self.turn} ---")

            # Ход игрока
            self.player_turn()
            if not self.enemy.is_alive():
                break

            # Ход врага
            self.enemy_turn()

        # Результаты боя
        if self.player.is_alive():
            print(f"\n{self.player.name} побеждает {self.enemy.name}!")
            self.player.experience += self.enemy.exp_reward
            print(f"{self.player.name} получает {self.enemy.exp_reward} опыта!")
            self.player.check_level_up()
            self.player.restore_stats()
            return True
        else:
            print(f"\n{self.player.name} был побежден {self.enemy.name}!")
            return False

    def player_turn(self):
        """Ход игрока"""
        # В демо-версии выбираем случайное действие
        action_type = random.choice(["attack", "ability", "item"])

        if action_type == "attack" or not self.player.abilities:
            # Базовая атака
            damage = self.player.basic_attack()
            print(f"{self.player.name} атакует {self.enemy.name} и наносит {damage} урона!")
            actual_damage = self.enemy.take_damage(damage)
            if actual_damage != damage:
                print(f"{self.enemy.name} блокирует часть урона и получает только {actual_damage} урона!")

        elif action_type == "ability" and self.player.abilities:
            # Использование способности
            ability = random.choice(self.player.abilities)
            if ability.is_ready() and self.player.mana >= ability.mana_cost:
                if isinstance(self.player, Mage) or isinstance(self.player, Paladin):
                    # Для мага и паладина выбираем случайную способность
                    ability_index = random.randint(0, len(self.player.abilities)-1)
                    damage = self.player.special_ability(ability_index)
                else:
                    damage = self.player.special_ability()

                if damage > 0:
                    print(f"{self.player.name} использует {ability.name} и наносит {damage} урона!")
                    actual_damage = self.enemy.take_damage(damage)
                    if actual_damage != damage:
                        print(f"{self.enemy.name} блокирует часть урона и получает только {actual_damage} урона!")
                elif damage < 0:
                    print(f"{self.player.name} использует {ability.name} и восстанавливает {-damage} здоровья!")
            else:
                print(f"{self.player.name} пытается использовать {ability.name}, но не хватает маны или способность на кулдауне!")
                # Если способность не готова, просто атакуем
                damage = self.player.basic_attack()
                print(f"{self.player.name} атакует {self.enemy.name} и наносит {damage} урона!")
                actual_damage = self.enemy.take_damage(damage)
                if actual_damage != damage:
                    print(f"{self.enemy.name} блокирует часть урона и получает только {actual_damage} урона!")

        elif action_type == "item" and self.player.inventory:
            # Использование предмета
            item = random.choice(self.player.inventory)
            if item.type == "potion":
                self.player.use_item(self.player.inventory.index(item))
            else:
                print(f"{self.player.name} решает не использовать предметы в этот раз.")
                damage = self.player.basic_attack()
                print(f"{self.player.name} атакует {self.enemy.name} и наносит {damage} урона!")
                actual_damage = self.enemy.take_damage(damage)
                if actual_damage != damage:
                    print(f"{self.enemy.name} блокирует часть урона и получает только {actual_damage} урона!")

        # Уменьшаем кулдауны
        self.player.reduce_cooldowns()

    def enemy_turn(self):
        """Ход врага"""
        # Враг выбирает действие случайным образом
        if random.random() < 0.7 or not self.enemy.abilities:  # 70% шанс на базовую атаку
            damage = self.enemy.basic_attack()
            print(f"{self.enemy.name} атакует {self.player.name} и наносит {damage} урона!")
        else:  # 30% шанс на способность
            damage = self.enemy.special_ability()
            if damage > 0:
                print(f"{self.enemy.name} использует способность и наносит {damage} урона!")

        # Применяем урон к игроку
        if damage > 0:
            actual_damage = self.player.take_damage(damage)
            if actual_damage != damage:
                print(f"{self.player.name} блокирует часть урона и получает только {actual_damage} урона!")

        # Уменьшаем кулдауны врага
        self.enemy.reduce_cooldowns()

# Демонстрация работы системы
if __name__ == "__main__":
    print("=== Демонстрация ролевой игры ===")

    # Создаем героев
    warrior = Warrior("Арагорн")
    mage = Mage("Гэндальф")
    rogue = Rogue("Леголас")
    paladin = Paladin("Тирион")

    # Добавляем предметы
    warrior.add_item(Item("Зелье здоровья", "potion", 30))
    mage.add_item(Item("Посох огня", "weapon", 10))
    rogue.add_item(Item("Ядовитый кинжал", "weapon", 15))
    paladin.add_item(Item("Щит света", "armor", 8))

    # Выбираем случайного героя для демонстрации
    hero = random.choice([warrior, mage, rogue, paladin])
    print(f"\nСоздан герой: {hero}")
    print("Способности:")
    for ability in hero.abilities:
        print(f"- {ability}")
    print("Предметы:")
    for item in hero.inventory:
        print(f"- {item}")

    # Создаем врагов разного уровня
    enemies = [Goblin(), Orc(), Dragon()]

    # Проводим несколько боев
    for enemy in enemies:
        battle = Battle(hero, enemy)
        victory = battle.start()

        if not victory:
            print("Игра окончена!")
            break

        input("\nНажмите Enter для продолжения...")

    print("\n=== Демонстрация завершена ===")

=== Демонстрация ролевой игры ===

Создан герой: Арагорн (Ур. 1) - Здоровье: 120/120, Мана: 30/30
Способности:
- Мощный удар (урон: 25, мана: 15, кулдаун: 2)
Предметы:
- Зелье здоровья (potion, значение: 30, прочность: 100%)

=== Бой начинается: Арагорн vs Гоблин ===
Арагорн (Ур. 1) - Здоровье: 120/120, Мана: 30/30
Гоблин (Ур. 1) - Здоровье: 40/40, Мана: 0/0


--- Ход 1 ---
Арагорн использует Мощный удар и наносит 28 урона!
Гоблин блокирует часть урона и получает только 27 урона!
Гоблин атакует Арагорн и наносит 8 урона!
Арагорн блокирует часть урона и получает только 3 урона!

--- Ход 2 ---
Арагорн атакует Гоблин и наносит 14 урона!
Гоблин блокирует часть урона и получает только 13 урона!

Арагорн побеждает Гоблин!
Арагорн получает 50 опыта!

Нажмите Enter для продолжения...stop

=== Бой начинается: Арагорн vs Орк ===
Арагорн (Ур. 1) - Здоровье: 120/120, Мана: 30/30
Орк (Ур. 1) - Здоровье: 80/80, Мана: 0/0


--- Ход 1 ---
Арагорн использует Мощный удар и наносит 28 урона!
Орк блокируе